# search-cn-tg-message

a useful tool to search cn chat history in tg

## create env

```
conda create -n bs4 python=3.8 beautifulsoup4
python -m ipykernel install --user --name bs4 --display-name "Python (bs4)"
```

## begining

### download tg message 

export chat history, uncheck all multimedia

### put this file into the download dir

### set up config

### update back_list using out.txt

1. find frequent word in [图悦-在线词频分析工具-词云图制作软件](http://www.picdata.cn/picdata/), download xls
2. copy-paste unwanted topic word in back_list from xls
3. repeat 1,2 until the most freq is low(like < 4)

## output

export plain text to out.txt



In [ ]:
# config
USER_TO_FIND = "SomeHandleName"

In [79]:
from bs4 import BeautifulSoup

In [165]:
def test_user_to_find(msg):
    
    # message find by user
    
    txt = msg.find("div", class_="from_name")
    if not hasattr(txt, "text"): return False
    return USER_TO_FIND in txt.text.strip()

def parse2json(msg):
    
    # parse xml_structure to handy json
    
    txt = msg.find("div", class_="text")
    if not hasattr(txt, "text"): txt = ""
    else :
        txt = txt.text.strip()
        
    return {
        "date":msg.find("div", class_="date")["title"],
        "text":txt
    }
    
def get_list_by_file(filename):
    
    # iterate through file, 
    # apply the two filter func above
    # convert structure, xml to json
    
    bs = BeautifulSoup(open(filename, encoding="UTF-8"))
    
    # filter files by date
#     time = bs.find("div", class_="body")
#     time = time.text.strip()
#     [day, mon, ] = time.split()[:2]
#     if not ( int(day) >= 23 and mon == "July" ):
#         return []
    
    body_l = bs.find_all("div", class_="body")
    body_l = list(filter(lambda msg:msg["class"].__len__()==1, body_l))
    body_l = list(filter(test_user_to_find, body_l))
    body_l = list(map(parse2json, body_l))
    return body_l 


In [166]:
# get input html file names

import os
filenames = os.listdir('./')
filenames = list(filter( lambda name: ".html" in name, filenames ,))
obj_list = []
for name in filenames:
    obj_list.extend(get_list_by_file(name))
    

In [1]:
black_list = """
额度
苹果
域名
证书
服务器
源站
win7
cdn
机器人
cloudflare
激活
老外
显卡
cpu
折腾
移动网络
hexo
老王
恶心
telegram
静态
"""
black_list = black_list.split()
black_list 

['额度',
 '苹果',
 '域名',
 '证书',
 '服务器',
 '源站',
 'win7',
 'cdn',
 '机器人',
 'cloudflare',
 '激活',
 '老外',
 '显卡',
 'cpu',
 '折腾',
 '移动网络',
 'hexo',
 '老王',
 '恶心',
 'telegram',
 '静态']

In [185]:
# output to file
# filter text by min_len 2
# filter text by back_list

from functools import partial

txt_list =  obj_list
txt_list = list(filter(lambda x:len(x["text"])>=2, txt_list))

for black_word in black_list:
    txt_list = list(filter(lambda x:black_word not in (x["text"]), txt_list))

fp = open("out.txt", "w", encoding="UTF-8")

p = partial(print, file=fp)
for index, txt in enumerate(txt_list ):
    p( txt["text"],txt["date"], index,)
    
fp.close()